In [1]:
import sys
sys.path.append('..')

from adat.models.classification_model import LogisticRegressionOnTfIdf
from sklearn.metrics import roc_auc_score

In [2]:
train = []
train_labs = []
with open('data/train.text') as f1, open('data/train.labels') as f2:
    for l1, l2 in zip(f1, f2):
        train.append(l1.strip())
        train_labs.append(int(l2.strip()))

In [3]:
test = []
test_labs = []
with open('data/test.text') as f1, open('data/test.labels') as f2:
    for l1, l2 in zip(f1, f2):
        test.append(l1.strip())
        test_labs.append(int(l2.strip()))

In [4]:
model = LogisticRegressionOnTfIdf()

In [5]:
model.fit(train, train_labs)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [51]:
probs = model.predict(test)

In [52]:
metric = roc_auc_score(y_true=test_labs, y_score=probs[:, 1])
print(f'ROC AUC = {metric}')

ROC AUC = 0.8702421297428788


In [53]:
from sklearn.metrics import f1_score

In [55]:
best_f1 = 0

for thres in np.linspace(0, 1):
    y_pred = (probs[:, 1] > thres).astype(int)
    f1 = f1_score(test_labs, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thres = thres
        
print(f'F1 = {best_f1} Thres = {best_thres}')

F1 = 0.1841509433962264 Thres = 0.061224489795918366


## Adversarial examples

In [20]:
import pandas as pd
import numpy as np

In [10]:
# all examples here are positive
mcmc_output = pd.read_csv('mcmc_output.csv')

In [48]:
# mcmc_output.head()

In [56]:
unique_postitive_examples = mcmc_output.original.unique()

In [89]:
unique_postitive_examples.shape

(3121,)

In [57]:
probs = model.predict(unique_postitive_examples)

In [59]:
y_pred = (probs[:, 1] < best_thres).astype(int)
y_true = np.ones(len(unique_postitive_examples))

In [60]:
f1_score(y_true, y_pred)

0.7839470095460744

In [90]:
generated = mcmc_output.loc[(mcmc_output.prob_diff <= -0.3) & \
                            (mcmc_output.bleu_diff >= -0.1), 'generated_sequence'].values

In [91]:
generated.shape

(9267,)

In [92]:
probs = model.predict(generated)

In [93]:
y_pred = (probs[:, 1] < best_thres).astype(int)
y_true = np.ones(len(generated))

In [94]:
f1_score(y_true, y_pred)

0.7552719946272666